#### Native Point Map Visualization

Point map visualization is natively supported, if your table outputs longitude-latitude pairs. It is recommended to limit the query to not too many rows for performance reasons.

::: {.callout-note}

As of 2025-08-20, GEOMETRY/GEOGRAPHY columns cannot be returned in a notebook yet, therefore instead of `select *`, we temporarily use the pattern, for a column `g` of type GEOMETRY/GEOGRAPHY:

```sql
select
  * except (g),
  st_asewkt(g) as ewkt_g
```

:::

In [0]:
%sql
create
or replace table tmp_geometries as
select
  "Onze Lieve Vrouwetoren" as name,
  st_transform(st_point(155000, 463000, 28992), 4326) as geometry
union all
select
  "Bermuda Triangle" as name,
  st_makepolygon(
    st_makeline(
      array(
        st_point(- 80.1935973, 25.7741566, 4326),
        st_point(- 64.7563086, 32.3040273, 4326),
        st_point(- 66.1166669, 18.4653003, 4326),
        st_point(- 80.1935973, 25.7741566, 4326)
      )
    )
  ) as geometry;

select
  * except (geometry),
  st_asewkt(geometry) as ewkt_geometry
from
  tmp_geometries

Our example table contains not only points, but the native visualization can only show points, so one thing we can do is to show the centroids.

In the above example, the SRID happens to be 4326, therefore the below `st_transform` is a unnecessary. In general, however, you'd want to transform the geometries to 4326 in order to obtain valid `(lon, lat)` tuples.

In [0]:
%sql
with t as (
  select
    name,
    st_transform(st_centroid(geometry), 4326) centroid,
    st_x(centroid) lon,
    st_y(centroid) lat
  from
    tmp_geometries
)
select
  * except (centroid),
  st_asewkt(centroid) as ewkt_centroid
from
  t

Databricks visualization. Run in Databricks to view.

Once the result above is shown, click on the `+` icon right of the `Table` tab to add the visualization "Map (Markers)" such as the one shown on the below image.

![native_pointmap](img/native_pointmap.png)

For another, larger example of native map visualization, see [the Overture example used in the DuckDB notebook](../duckdb-on-databricks/DuckDB_on_Databricks.ipynb).